In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import coo_matrix, hstack
from sklearn.preprocessing import LabelEncoder
import re
from tqdm import tqdm

In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)


def fillna(df,col):
    no_inf = df[(df[col] != np.PINF)&(df[col] != np.NINF)&(df[col] != np.Inf)][col]
    df_max = max(no_inf)
    df_min = min(no_inf)
    df[col] = df[col].replace(np.PINF,df_max)
    df[col] = df[col].replace(np.NINF,df_min)
    df[col] = df[col].replace(np.inf,df_max)
    df[col] = df[col].fillna(df[col].mean())
    return df[col]


def view_na_portion(df,col):
    return sum(df[col].isnull())/len(df)


def one_hot_encoding(df,col,pre_fix,drop=True):
    df = df.copy()
    df[col] = df[col].fillna('NA_NOT_FOUND')
    col_name_list = []
    print('before encoding, shape is: {}'.format(df.shape))
    for each in df[col].unique():
        name = str(each)
        col_name = pre_fix + '_'+ name.replace(' ','_')
        col_name_list.append(col_name)
        df[col_name] = 0
        df.loc[df[col]==each,col_name] = 1
    if drop:
        df = df.drop([col],axis=1)
    print('after encoding, shape is: {}'.format(df.shape))
    return df,col_name_list
    

class ordinal_encoder:
    def __init__(self,fillna='NAN_FILL'):
        self.fillna = fillna
    
    def fit(self,sr,NA_VALUE=None,realNA2Nega1=True):
        """
        spycify the value which has already been encoded. make it to -1 after encoding
        """
        self.NA_VALUE=NA_VALUE
        order = sr.fillna(self.fillna).value_counts()
        if self.NA_VALUE is not None:
            order[NA_VALUE] = 9999999

        if realNA2Nega1:
            order[self.fillna] = 99999999
        order = order.sort_values(ascending=True)


        self.mapping = pd.Series(index=order.index.values, data=list(range(1,len(order)+1)))
        if self.NA_VALUE is not None:
            self.mapping[NA_VALUE] = -1

        if realNA2Nega1:
            self.mapping[self.fillna] = -1
    
    def transform(self,sr):
        return sr.fillna(self.fillna).map(self.mapping)

In [3]:
oe = ordinal_encoder()

In [4]:
X_train = pd.read_pickle(ENV.lightgbm_train_764.value)
print(X_train.shape)

X_test = pd.read_pickle(ENV.lightgbm_test_764.value)
print(X_test.shape)

X = pd.concat([X_train.drop('TARGET',axis=1),X_test])
print(X.shape)

(307511, 764)
(48744, 763)
(356255, 763)


# Scan Category col

In [5]:
col_list = []
count_list = []
thread = 100
for col in X.columns:
    num = X[col].nunique()
    if num <= thread:
        col_list.append(col)
        count_list.append(num)
print(len(col_list))
seri = pd.Series(data = count_list,index=col_list)

69


In [6]:
cat_col = ['OCCUPATION_TYPE_Core staff',
           'ORGANIZATION_TYPE_School',
           'ORGANIZATION_TYPE_Transport: type 3',
           'ORGANIZATION_TYPE_Military',
           'FLAG_DOCUMENT_3_x',
           'FLAG_DOCUMENT_18_x',
           'ORGANIZATION_TYPE_Construction',
           'ORGANIZATION_TYPE_Self-employed',
           'NAME_INCOME_TYPE_State servant',
           'WALLSMATERIAL_MODE_Stone, brick',
           'OCCUPATION_TYPE_Drivers',
           'FLAG_DOCUMENT_16_x',
           'WEEKDAY_APPR_PROCESS_START_MONDAY',
           'OCCUPATION_TYPE_Medicine staff',
           'FLAG_DOCUMENT_16_x',
           'WEEKDAY_APPR_PROCESS_START_MONDAY',
           'OCCUPATION_TYPE_Medicine staff',
           'FLAG_WORK_PHONE_x',
           'REG_CITY_NOT_LIVE_CITY_x',
           'REGION_RATING_CLIENT_x',
           'CODE_GENDER_x',
           'unique_CODE_GENDER_NAME_EDUCATION_TYPE',
           'CONTACT',
           'NAME_HOUSING_TYPE_x',
           'NAME_FAMILY_STATUS_x',
           'prev_WEEKDAY_APPR_PROCESS_START_max',
           'WEEKDAY_APPR_PROCESS_START_x',
           'CAT_MONTHS_BIRTH',
           'OCCUPATION_TYPE_x',
           'bureau_onehot_CREDIT_TYPE_Credit card',
           'bureau_onehot_CREDIT_ACTIVE_Active',
           'REFUSED_HOUR_APPR_PROCESS_START_MAX',
           'HOUR_APPR_PROCESS_START_x',
           'HOUR_APPR_PROCESS_START_y',
           'REFUSED_HOUR_APPR_PROCESS_START_MIN',
           'prev_onehot_NAME_SELLER_INDUSTRY_Connectivity',
           'prev_onehot_NAME_YIELD_GROUP_2.0',
           'bureau_onehot_CREDIT_TYPE_Microloan',
           'prev_onehot_NAME_TYPE_SUITE_nan',
           'ORGANIZATION_TYPE_x']

numerical_col = list(set(X.columns) - set(cat_col))

# FillNA

## Part 1. Numberical

In [7]:
for each in tqdm(numerical_col):
    print(each)
    X[each] = fillna(X,each)
#     X[each] = X[each].fillna(X[each].mean())
    
# col = 'bureau_debt_credit_ratio'
# X[col] = X[col].replace(np.NINF,-176)
# X[col] = X[col].replace(np.PINF,8)
# X[col] = X[col].fillna(X[col].mean())

  0%|          | 0/726 [00:00<?, ?it/s]

last_10by50_fraction_installment_paid_late_mean


  0%|          | 1/726 [00:01<15:18,  1.27s/it]

prev_AMT_ANNUITY_max


  0%|          | 2/726 [00:02<12:19,  1.02s/it]

...

100%|█████████▉| 724/726 [09:24<00:01,  1.28it/s]

AMT_ANNUITY_divide_OWN_CAR_AGE


100%|█████████▉| 725/726 [09:24<00:00,  1.28it/s]

prev_AMT_ANNUITY_divide_DAYS_FIRST_DUE_min


100%|██████████| 726/726 [09:25<00:00,  1.28it/s]


## Part 2. Categorical

In [8]:
import re
for each in cat_col:
    X[each] = X[each].astype(str)
    prefix = re.sub(r'[ _:,.]','',each)
    print(prefix)
    X[each] = X[each].apply(lambda x: prefix+x)

OCCUPATIONTYPECorestaff
ORGANIZATIONTYPESchool
ORGANIZATIONTYPETransporttype3
ORGANIZATIONTYPEMilitary
FLAGDOCUMENT3x
FLAGDOCUMENT18x
ORGANIZATIONTYPEConstruction
ORGANIZATIONTYPESelf-employed
NAMEINCOMETYPEStateservant
WALLSMATERIALMODEStonebrick
OCCUPATIONTYPEDrivers
FLAGDOCUMENT16x
WEEKDAYAPPRPROCESSSTARTMONDAY
OCCUPATIONTYPEMedicinestaff
FLAGDOCUMENT16x
WEEKDAYAPPRPROCESSSTARTMONDAY
OCCUPATIONTYPEMedicinestaff
FLAGWORKPHONEx
REGCITYNOTLIVECITYx
REGIONRATINGCLIENTx
CODEGENDERx
uniqueCODEGENDERNAMEEDUCATIONTYPE
CONTACT
NAMEHOUSINGTYPEx
NAMEFAMILYSTATUSx
prevWEEKDAYAPPRPROCESSSTARTmax
WEEKDAYAPPRPROCESSSTARTx
CATMONTHSBIRTH
OCCUPATIONTYPEx
bureauonehotCREDITTYPECreditcard
bureauonehotCREDITACTIVEActive
REFUSEDHOURAPPRPROCESSSTARTMAX
HOURAPPRPROCESSSTARTx
HOURAPPRPROCESSSTARTy
REFUSEDHOURAPPRPROCESSSTARTMIN
prevonehotNAMESELLERINDUSTRYConnectivity
prevonehotNAMEYIELDGROUP20
bureauonehotCREDITTYPEMicroloan
prevonehotNAMETYPESUITEnan
ORGANIZATIONTYPEx


# Scan Na

In [9]:
scan_nan_portion(X).sort_values()

AMT_CREDIT_divide_AMT_ANNUITY                                                    0.0
50_period_trend_installment_paid_over_amount                                     0.0
creditcard_AMT_BALANCE_divide_AMT_CREDIT_LIMIT_ACTUAL_median                     0.0
ORGANIZATION_TYPE_Self-employed                                                  0.0
PREV_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_MEAN                                   0.0
last_10_installment_paid_late_in_days_median                                     0.0
REFUSED_AMT_ANNUITY_MAX                                                          0.0
inst_RATE_LATE_DAYS_SIGN_sum                                                     0.0
prev_AMT_ANNUITY_divide_DAYS_FIRST_DRAWING_minus_DAYS_LAST_DUE_min               0.0
NAME_FAMILY_STATUS_NAME_EDUCATION_TYPE_CNT_CHILDREN_max_diff                     0.0
PREV_CODE_REJECT_REASON_SCO_MEAN                                                 0.0
last_loan_installment_paid_late_count                            

# Saving

In [10]:
train = X.iloc[:307511].copy()
train['TARGET'] = X_train['TARGET'].values
test = X.iloc[307511:].copy()

In [11]:
print(train.shape)
print(test.shape)

(307511, 764)
(48744, 763)


In [13]:
train.to_pickle(ENV.lightgbm_train_764_nn.value)
test.to_pickle(ENV.lightgbm_test_764_nn.value)